In [292]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import pickle
import re

from konlpy.tag import Okt

In [293]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [294]:
chatbot_data = pd.read_csv('./train.csv')
chatbot_data.head(3)

,Q,A
0,보관만 하는데 9만 원인가요 락커는 얼마죠,대형은 28만 원 중형은 19만 원 소형은 15만 원입니다
1,#@소속#이요 이름이 정겹네요,그 식당에서 드셔 보시면 홍천하면 보리밥부터 생각나실 거예요
2,#@소속# 공원 근처에 맛있는 분식집 추천해 주세요,국물 떡볶이가 있는 #@소속#을 추천해 드립니다


In [295]:
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [296]:
# 데이터 개수
len(question), len(answer)

(33994, 33994)

In [297]:
# 일부만 학습에 사용
question = question[:1000]
answer = answer[:1000]

for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 보관만 하는데 9만 원인가요 락커는 얼마죠 
A : 대형은 28만 원 중형은 19만 원 소형은 15만 원입니다 

Q : #@소속#이요 이름이 정겹네요 
A : 그 식당에서 드셔 보시면 홍천하면 보리밥부터 생각나실 거예요 

Q : #@소속# 공원 근처에 맛있는 분식집 추천해 주세요 
A : 국물 떡볶이가 있는 #@소속#을 추천해 드립니다 

Q : 실내에도 다양하게 준비되어 있네요 
A : 그러니 직접 체험해 보시길 바랍니다 

Q : 이번에 김포 여행을 가려고 하는데요 
A : 무엇을 도와드릴까요 



In [337]:
question

['보관 만 하는데 9만 원 인가요 락커 는 얼마 죠',
 '#@ 소속 # 이요 이름 이 정겹네요',
 '#@ 소속 # 공원 근처 에 맛있는 분식집 추천 해 주세요',
 '실내 에도 다양하게 준비 되어 있네요',
 '이번 에 김포 여행 을 가려고 하는데요',
 '아무 것 도 없는 줄 알았어요',
 '#@ 소속 # 맞습니다',
 '배 를 타는 시간 이 있나요',
 '절매 나루터 에서 진 탄 나루터 까지 하는 코스 맞지요',
 '#@ 소속 # 에서 받으면 되는 거 죠',
 '그건 따로 못 보긴 했는데 수요일 만 쉬는 건가 요',
 '영월 10 경 중 에 사람 이 지은 건축물 같은 거 말구요 강가 따라서 풍경 볼 수 있는 곳 추천 좀 해주세요',
 '#@ 소속 # 예약 할 수 있나 확인 하려구요',
 '녹두 한 방 백숙 은 가격 이 얼마 예요',
 '건물 뒤 에 있어서 주차장 이 안 보였군요 앞 에 차 들 은 뭐 예요',
 '체험 할 수 있는 프로그램 이 있습니까',
 '친절한 안내 감사 드립니다',
 '강습 은 받지 않고 장비 렌탈 만 이용 할 수 는 없나요',
 '땀 이 많은 편이 라 복장 때문 에 문의 드려요 등산 복 긴 팔 에 구스 경량패딩 을 입고 갈려고 하 거든요',
 '제 가 관심 은 있는데 전혀 할 줄 몰라서요',
 '제 이메일 이 요 #@ 계정 # 입니다',
 '40분 이나 기다려야 하네요',
 '수상스키 를 체험 하고 싶은데요 금액 이 어떻게 되나요',
 '이천 여행 중 인데 맛집 을 찾고 있어요 영업 시간 이 어떻게 되나요',
 '안녕하세요 저희 가 오늘 여행 마지막 날인데요 불꽃 축제 보고 엑스포역 에서 바로 기차 를 타야 하 거든요',
 '진해 군항제 를 다녀와야겠네요',
 '엑스포 공원 갔다가 돌 머리 해수욕장 들러서 해수 찜 하러 갈거에요',
 '방금 예약 했던 게 취소 가 되서 확인 좀 부탁 드리려 구요',
 '주말 에 단독 으로 요트 투어 가 가능하나요',
 '대표 메뉴 가 무엇 인가요',
 '초등학생 체험학습 도 있나요',
 '공룡 

In [344]:
# 형태소 분석 함수
def pos_tag(sentences):

    tagger = Okt()

    # 문장 품사 변수 초기화
    sentences_pos = []

    # 모든 문장 반복
    for sentence in sentences:
        sentence = re.sub(RE_FILTER, "", sentence)

        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [345]:
question = pos_tag(question)
answer = pos_tag(answer)

In [346]:
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 보관 만 하는데 9만 원 인가요 락커 는 얼마 죠
A : 대형 은 28만 원 중형 은 19만 원 소형 은 15만 원 입니다

Q : #@ 소속 # 이요 이름 이 정겹네요
A : 그 식당 에서 드셔 보시 면 홍천 하면 보리밥 부터 생각나실 거 예요

Q : #@ 소속 # 공원 근처 에 맛있는 분식집 추천 해 주세요
A : 국물 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다

Q : 실내 에도 다양하게 준비 되어 있네요
A : 그러니 직접 체험 해 보시길 바랍니다

Q : 이번 에 김포 여행 을 가려고 하는데요
A : 무엇 을 도 와 드릴 까요



In [347]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

In [348]:
print(words)

['보관', '만', '하는데', '9만', '원', '인가요', '락커', '는', '얼마', '죠', '#@', '소속', '#', '이요', '이름', '이', '정겹네요', '#@', '소속', '#', '공원', '근처', '에', '맛있는', '분식집', '추천', '해', '주세요', '실내', '에도', '다양하게', '준비', '되어', '있네요', '이번', '에', '김포', '여행', '을', '가려고', '하는데요', '아무', '것', '도', '없는', '줄', '알았어요', '#@', '소속', '#', '맞습니다', '배', '를', '타는', '시간', '이', '있나요', '절매', '나루터', '에서', '진', '탄', '나루터', '까지', '하는', '코스', '맞지요', '#@', '소속', '#', '에서', '받으면', '되는', '거', '죠', '그건', '따로', '못', '보긴', '했는데', '수요일', '만', '쉬는', '건가', '요', '영월', '10', '경', '중', '에', '사람', '이', '지은', '건축물', '같은', '거', '말구요', '강가', '따라서', '풍경', '볼', '수', '있는', '곳', '추천', '좀', '해주세요', '#@', '소속', '#', '예약', '할', '수', '있나', '확인', '하려구요', '녹두', '한', '방', '백숙', '은', '가격', '이', '얼마', '예요', '건물', '뒤', '에', '있어서', '주차장', '이', '안', '보였군요', '앞', '에', '차', '들', '은', '뭐', '예요', '체험', '할', '수', '있는', '프로그램', '이', '있습니까', '친절한', '안내', '감사', '드립니다', '강습', '은', '받지', '않고', '장비', '렌탈', '만', '이용', '할', '수', '는', '없나요', '땀', '이', '많은', '편이', '라', '복장', '때문',

In [349]:
# 길이가 0인 단어 삭제
words = [word for word in words if len(word) > 0]

In [350]:
# 중복된 단어 삭제
words = list(set(words))

In [351]:
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [352]:
len(words)

3640

In [353]:
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '돌',
 '한식',
 '찾겠어서요',
 '군항제',
 '선택',
 '실제',
 '산골',
 '이동',
 '께',
 '각종',
 '계',
 '사무소',
 '평일',
 '등재',
 '사면',
 '바베큐',
 '노래',
 'F',
 '대학생',
 '4시간',
 '현금',
 '인실이',
 '레저',
 '계곡',
 '봐서요',
 '태워주고',
 '쾌적합니다',
 '훨',
 '가능하시구요',
 '괜찮나요',
 '10월',
 '절대',
 '동',
 '많은데',
 '없이',
 '나무숲',
 '일이',
 '선',
 '2만원',
 '걸릴',
 '빠르실',
 '불가능합니다',
 '받으셨던',
 '하시길',
 '6시',
 '복층',
 '카페',
 '성동구',
 '잡히나요',
 '경로',
 '저녁',
 '힐링',
 '아무래도',
 '남원',
 '계좌',
 'MTB',
 '어쩔',
 '전체',
 '놀기에',
 '되네요',
 '일상생활',
 '냄새',
 '찾는데요',
 '소형',
 '학교',
 '2',
 '있으실까',
 '인프라',
 '등록증',
 '적어주신',
 '마세요',
 '기타',
 '12월',
 '42',
 '교',
 '드라이기',
 '봄',
 '드리자면요',
 '이라는',
 '현장',
 '대해',
 '못',
 '적',
 '동일합니다',
 '부족하지는',
 '이있습니다',
 '팔고',
 '캠핑장',
 '알려주시나요',
 '약제',
 '10900원',
 '되죠',
 '드릴려고요',
 '이군',
 '들어가게',
 '가파른',
 '전혀',
 '담양',
 '플리',
 '제로',
 '더워서요',
 '이번',
 '초정약수',
 '손질',
 '흥미로운',
 '들',
 '지리산',
 '동행인',
 '다례',
 '저렴한',
 '충분히',
 '있다가',
 '특산',
 '들었거든요',
 '스파게티',
 '오나요',
 '토치',
 '조그만',
 '건물',
 '빈',
 '하시는건',
 '방영',
 '부분',
 '닭갈

In [308]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [309]:
# 단어 => 인덱스
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '돌': 4,
 '한식': 5,
 '찾겠어서요': 6,
 '군항제': 7,
 '선택': 8,
 '실제': 9,
 '산골': 10,
 '이동': 11,
 '께': 12,
 '각종': 13,
 '계': 14,
 '사무소': 15,
 '평일': 16,
 '등재': 17,
 '사면': 18,
 '바베큐': 19,
 '노래': 20,
 'F': 21,
 '대학생': 22,
 '4시간': 23,
 '현금': 24,
 '인실이': 25,
 '레저': 26,
 '계곡': 27,
 '봐서요': 28,
 '태워주고': 29,
 '쾌적합니다': 30,
 '훨': 31,
 '가능하시구요': 32,
 '괜찮나요': 33,
 '10월': 34,
 '절대': 35,
 '동': 36,
 '많은데': 37,
 '없이': 38,
 '나무숲': 39,
 '일이': 40,
 '선': 41,
 '2만원': 42,
 '걸릴': 43,
 '빠르실': 44,
 '불가능합니다': 45,
 '받으셨던': 46,
 '하시길': 47,
 '6시': 48,
 '복층': 49,
 '카페': 50,
 '성동구': 51,
 '잡히나요': 52,
 '경로': 53,
 '저녁': 54,
 '힐링': 55,
 '아무래도': 56,
 '남원': 57,
 '계좌': 58,
 'MTB': 59,
 '어쩔': 60,
 '전체': 61,
 '놀기에': 62,
 '되네요': 63,
 '일상생활': 64,
 '냄새': 65,
 '찾는데요': 66,
 '소형': 67,
 '학교': 68,
 '2': 69,
 '있으실까': 70,
 '인프라': 71,
 '등록증': 72,
 '적어주신': 73,
 '마세요': 74,
 '기타': 75,
 '12월': 76,
 '42': 77,
 '교': 78,
 '드라이기': 79,
 '봄': 80,
 '드리자면요': 81,
 '이라는': 82,
 '현장': 83,
 '대해': 84,
 '못': 85

In [310]:
# 인덱스 => 단어
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '돌',
 5: '한식',
 6: '찾겠어서요',
 7: '군항제',
 8: '선택',
 9: '실제',
 10: '산골',
 11: '이동',
 12: '께',
 13: '각종',
 14: '계',
 15: '사무소',
 16: '평일',
 17: '등재',
 18: '사면',
 19: '바베큐',
 20: '노래',
 21: 'F',
 22: '대학생',
 23: '4시간',
 24: '현금',
 25: '인실이',
 26: '레저',
 27: '계곡',
 28: '봐서요',
 29: '태워주고',
 30: '쾌적합니다',
 31: '훨',
 32: '가능하시구요',
 33: '괜찮나요',
 34: '10월',
 35: '절대',
 36: '동',
 37: '많은데',
 38: '없이',
 39: '나무숲',
 40: '일이',
 41: '선',
 42: '2만원',
 43: '걸릴',
 44: '빠르실',
 45: '불가능합니다',
 46: '받으셨던',
 47: '하시길',
 48: '6시',
 49: '복층',
 50: '카페',
 51: '성동구',
 52: '잡히나요',
 53: '경로',
 54: '저녁',
 55: '힐링',
 56: '아무래도',
 57: '남원',
 58: '계좌',
 59: 'MTB',
 60: '어쩔',
 61: '전체',
 62: '놀기에',
 63: '되네요',
 64: '일상생활',
 65: '냄새',
 66: '찾는데요',
 67: '소형',
 68: '학교',
 69: '2',
 70: '있으실까',
 71: '인프라',
 72: '등록증',
 73: '적어주신',
 74: '마세요',
 75: '기타',
 76: '12월',
 77: '42',
 78: '교',
 79: '드라이기',
 80: '봄',
 81: '드리자면요',
 82: '이라는',
 83: '현장',
 84: '대해',
 85: '못'

In [311]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [312]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)
x_encoder[0]

array([1318, 1999, 2252, 1820, 3464, 3390, 3149, 1412, 1298, 3582,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [313]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)
x_decoder[0]

array([   1,  444, 1113, 1772, 3464,  532, 1113, 3001, 3464,   67, 1113,
       2396, 3464,  732,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [314]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)
y_decoder[0]

array([ 444, 1113, 1772, 3464,  532, 1113, 3001, 3464,   67, 1113, 2396,
       3464,  732,    2,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [315]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)), dtype=np.int8)

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [316]:
# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

encoder_inputs = layers.Input(shape=(None,))

encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)
# recurrent_dropout : 순환 드롭아웃 - 과거가 아닌 현재 Input에 영향을 받는 Parameter에만 Dropout 적용

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

In [317]:
# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [318]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [319]:
# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

In [320]:
# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [321]:
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [322]:
# 훈련 시작
history = model.fit([x_encoder, x_decoder],
                    y_decoder,
                    epochs=1500,
                    batch_size=64)

Epoch 1/1500
16/16 [==============================] - 6s 156ms/step - loss: 5.3092 - acc: 0.5966
Epoch 2/1500
16/16 [==============================] - 3s 169ms/step - loss: 2.8671 - acc: 0.6377
Epoch 3/1500
16/16 [==============================] - 3s 172ms/step - loss: 2.6525 - acc: 0.6377
Epoch 4/1500
16/16 [==============================] - 3s 179ms/step - loss: 2.5531 - acc: 0.6388
Epoch 5/1500
16/16 [==============================] - 3s 187ms/step - loss: 2.4237 - acc: 0.6421
Epoch 6/1500
16/16 [==============================] - 3s 189ms/step - loss: 2.3307 - acc: 0.6461
Epoch 7/1500
16/16 [==============================] - 3s 195ms/step - loss: 2.2870 - acc: 0.6565
Epoch 8/1500
16/16 [==============================] - 3s 207ms/step - loss: 2.2475 - acc: 0.6655
Epoch 9/1500
16/16 [==============================] - 3s 209ms/step - loss: 2.2142 - acc: 0.6737
Epoch 10/1500
16/16 [==============================] - 3s 202ms/step - loss: 2.1862 - acc: 0.6768
Epoch 11/1500
16/16 [========

KeyboardInterrupt: 

In [323]:
# 문장 예측 테스트
input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
results = model.predict([input_encoder, input_decoder])

# 결과의 원핫인코딩 형식을 인덱스로 변환
indexs = np.argmax(results[0], 1) 

# 인덱스를 문장으로 변환
sentence = convert_index_to_text(indexs, index_to_word)
print(sentence)

1/1 [==============================] - 0s 292ms/step
국물 떡볶이 가 있는 #@ 소속 # 을 추천 해 드립니다 


In [324]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL) 

In [325]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)    

In [326]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [327]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [360]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('문어도 구워먹을 수 있나요')

# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 12ms/step


'저희 는 문어 와 함께 닭갈비 도 맥 반석 에 구워 드실 수 있습니다 '

In [361]:
input_seq = make_predict_input('승마체험 정보 알려주세요')

sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 14ms/step


'승마 체험 어떤 점 이 궁금하신 걸까 요 '

In [365]:
input_seq = make_predict_input('언제 영업하나요')

sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 16ms/step


'인터넷 예약 과 당일 현장 접수 가 가능 합니다 '